# this script is used to evaluate NPMI>0 result

In [1]:
import ast
import matplotlib.pyplot as plt
import csv
import xlrd
import matplotlib

In [2]:
#define dic
# input file dir
NPMI_sort_ngp_and_NPMI_xlsx_dic = '.\\inputfile_for_analysis\\NPMI_by_threshold.xlsx'
genes_to_phenotype_dic='.\\inputfile_for_analysis\\genes_to_phenotype.txt'
# output file dir
statistic_dic='.\\statistic_result\\'
evaluation_pair_csv_dic = '.\\evaluation_result\\evaluation_result.csv'

In [3]:
# figure setting
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [4]:
# read HPO file
genes_to_phenotype_list=[]
with open(genes_to_phenotype_dic,"r",encoding="utf-8") as f: 
    for line in f.readlines():#readlines以列表输出文件内容
        genes_to_phenotype_list.append(line.strip('\n').split('\t'))
        
del genes_to_phenotype_list[0]

genes_to_phenotype_dict={}
for line in genes_to_phenotype_list:
    gene = line[1]
    phenotype_HP=line[2]
    if gene not in genes_to_phenotype_dict.keys():
        genes_to_phenotype_dict[gene] = []
    if phenotype_HP not in genes_to_phenotype_dict[gene]:
        genes_to_phenotype_dict[gene].append(phenotype_HP)



In [5]:
# get all HPO number
HPnum_list = []
for gene in genes_to_phenotype_dict:
    HPnum_list = HPnum_list+genes_to_phenotype_dict[gene]
    
HPnum_list =  list(set(HPnum_list))

In [6]:


# read NPMI file and do evaluation
data = xlrd.open_workbook(NPMI_sort_ngp_and_NPMI_xlsx_dic) 
# table = data.sheet_by_name('all pairs NPMI>0 ngp > 5')
table = data.sheet_by_name('all pairs NPMI >0')
data_dict = {}
for rowNum in range(table.nrows):
# if 去掉表头
    if rowNum > 0:
        if table.row_values(rowNum)[0] != '':
            if table.row_values(rowNum)[0] not in data_dict.keys():
                data_dict[table.row_values(rowNum)[0]] = []
            data_dict[table.row_values(rowNum)[0]].append( [table.row_values(rowNum)[1],#phenotype
                                                            table.row_values(rowNum)[2],#gene_sfari_class
                                                            table.row_values(rowNum)[3],#NPMI
                                                            table.row_values(rowNum)[4],#n_g
                                                            table.row_values(rowNum)[5],#n_p
                                                            table.row_values(rowNum)[6]]#n_gp
                                                          )

gene_number_dict = {'1':[[],[],[]],'2':[[],[],[]],'3':[[],[],[]],'S':[[],[],[]],'NA':[[],[],[]]}


# evaluation and write file
with open(evaluation_pair_csv_dic, 'w', encoding='utf-8-sig',newline='') as f:
    writer = csv.writer(f)
    # writer.writerow(["Gene is not in HPO = 2", "No HP number of phenotype = 3","Pair found in HPO = 0", "Pair not found in HPO = 1"])
    writer.writerow(["gene", "phenotype","gene_sfari_class", "NPMI","n_g","n_p","n_gp","if_found_in_HPO","if_gene_found_in_HPO"])
    for gene in data_dict:

        if_gene_found = 'False'
        gene_data = []
        for pair in data_dict[gene]:
            phenotype = pair[0]
            phenotype_HP = ast.literal_eval(phenotype)[3]
            if type(pair[1]) == float:
                pair[1] = int(pair[1])
            gene_sfari_class = pair[1]
            NPMI = pair[2]
            n_g = pair[3]
            n_p = pair[4]
            n_gp =pair[5]
            
            gene_number_dict[str(pair[1])][2].append(gene)

            if gene not in genes_to_phenotype_dict.keys():
                if_found_in_HPO = "Gene is not in HPO"
            elif phenotype_HP == "NULL":
                if_found_in_HPO = "No HP number of phenotype"   
            elif phenotype_HP not in HPnum_list:
                if_found_in_HPO = "Phenotype not in c2upper"  
            else:
                gene_number_dict[str(pair[1])][1].append(gene)
                
                if phenotype_HP in genes_to_phenotype_dict[gene]:
                    if_found_in_HPO = "Pair found in HPO"
                    if_gene_found = 'True'
                else:
                    if_found_in_HPO = "Pair not found in HPO"
                        

            gene_data.append([gene, phenotype,gene_sfari_class,NPMI,n_g,n_p,n_gp,if_found_in_HPO])
        
        # write if found in HPO
        for pair_data in gene_data:
            pair_data.append(if_gene_found)
            writer.writerows([tuple(pair_data)])
        if if_gene_found == 'True' :
            if gene in gene_number_dict[str(pair[1])][1]:
                gene_number_dict[str(pair[1])][0].append(gene)

In [7]:
# statistic of evaluated result

for label in gene_number_dict:
    for index,item in enumerate(gene_number_dict[label]):
        gene_number_dict[label][index] = list(set(item))
        
        
        
sum_num1 = 0
sum_num2 = 0
sum_num3 = 0
for label in gene_number_dict:
      sum_num1  += len(gene_number_dict[label][0])
      sum_num2  += len(gene_number_dict[label][1])   
      sum_num3  += len(gene_number_dict[label][2])  